In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flower5/dataset/DAMAGED/Damaged (179).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (151).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (12).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (131).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (104).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (253).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (67).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (42).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (59).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (270).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (267).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (242).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (256).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (40).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (106).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (132).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (233).JPG
/kaggle/input/flower5/dataset/DAMAGED/Damaged (170).J

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the pre-trained model (EfficientNetB0) without the top layer
base_model = keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Create a new top model
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(8, activation='softmax')(x)  # Adjust the number of classes

# Create the final model
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Load your dataset (replace with your dataset)
datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1./255)
train_data = datagen.flow_from_directory('/kaggle/input/flower5', target_size=(224, 224), batch_size=32, subset='training')
val_data = datagen.flow_from_directory('/kaggle/input/flower5', target_size=(224, 224), batch_size=32, subset='validation')

# Train the model
model.fit(train_data, validation_data=val_data, epochs=20)

# Unfreeze some layers and fine-tune
base_model.trainable = True
for layer in base_model.layers[:-40]:  # Unfreezing last 40 layers
    layer.trainable = False

# Compile again with a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
model.fit(train_data, validation_data=val_data, epochs=15)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 589 images belonging to 1 classes.
Found 147 images belonging to 1 classes.
Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.3171 - loss: 2.0425 - val_accuracy: 1.0000 - val_loss: 1.4945
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.7892 - loss: 1.4474 - val_accuracy: 1.0000 - val_loss: 0.7364
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9549 - loss: 0.6733 - val_accuracy: 1.0000 - val_loss: 0.2017
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9867 - loss: 0.2315 - val_accuracy: 1.0000 - val_loss: 0.0536
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9984 - loss: 0.0721 - val_accuracy: 1.0000 - val_loss: 0.0263
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 1.0000 - loss: 0.0338 - val_accuracy: 1.0000 - val_loss: 0.0168
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 1.0000 - loss: 0.0239 - val_accuracy: 1.0000 - va